In [6]:
import pandas as pd
import os
import numpy as np

from importlib import reload

import Classifier

In [4]:
ls

Classifier.py          baselines.py           pred.csv
TEST.ipynb             challengeFunctions.py  test_raph.ipynb
Untitled.ipynb         kernels.py
__pycache__/           main.py


In [9]:

data_dir = '../data/'
os.listdir(data_dir)
files = os.listdir(data_dir)
files

['Xte0.csv',
 'Xte0_mat50.csv',
 'Xte1.csv',
 'Xte1_mat50.csv',
 'Xte2.csv',
 'Xte2_mat50.csv',
 'Xtr0.csv',
 'Xtr0_mat50.csv',
 'Xtr1.csv',
 'Xtr1_mat50.csv',
 'Xtr2.csv',
 'Xtr2_mat50.csv',
 'Ytr0.csv',
 'Ytr1.csv',
 'Ytr2.csv']

## Using sequences directly

In [10]:
def load_data(n,t):
    """
    n: index of the dataset to be loaded
    t:type of data , 'train' or 'test'
    """
    if(t=='test'):
        path = data_dir+'Xte'+str(n) +'.csv'
        data = pd.read_csv(path,header=-1)
        X = data.as_matrix()
        return X
    if(t=='train'):
        xpath = data_dir+'Xtr' +str(n) +'.csv'
        ypath = data_dir+'Ytr' +str(n) +'.csv'
        data = pd.read_csv(xpath,header=-1)
        labels = pd.read_csv(data_dir+'Ytr0.csv')
        X = data.as_matrix()
        X = X.reshape(np.shape(X)[0]) # Sequences in String format
        Y = labels['Bound'].as_matrix()
        return X,Y
    else:
        print('unknown type of dataset')
            


In [11]:

X,Y=load_data(0,'train')

In [12]:
# binary encoding
D = {'A':[1,0,0,0],
    'C':[0,1,0,0],
    'T':[0,0,1,0],
    'G':[0,0,0,1]}

def encode(sequence):
    code=[]
    for l in list(sequence):
        code+=(D[l])
    return code

#X = np.array([encode(x) for x in X])
#X.shape

### Spectrum kernel

In [13]:
# A:1 ,C:2 , T:3, G:4
code_dict={
    'A':1,
    'C':2,
    'T':3,
    'G':4
}
def code(string):
    return sum([code_dict[s]*4**(len(s)-k+1) for k,s in enumerate(string) ])

def phi(x,k=3):
    xa=np.ravel(x) # ensure that second dimension of x is zero
    L=len(xa[0]) # each element of xa is a string
    n=np.shape(x)[0]
    
    p=sum([4*4**j for j in range(k) ]) # dimension of the sparse representation 
    
    occs=np.zeros((n,p))
    for i,s in enumerate(xa):
        for j in range(L-k):
            occs[i,code(s[j:j+k])-1]+=1
            
    return occs


def spectrum_kernel(s,t):
    phi1=phi(s)
    phi2=phi(t)
    return np.dot(phi1,phi2)/np.sqrt(np.dot(phi1,phi1)*np.dot(phi2,phi2))

Xtr0,Ytr0=load_data(0,'train')
Xtr1,Ytr1=load_data(1,'train')
Xtr2,Ytr2=load_data(2,'train')



Xte0=load_data(0,'test')
Xtr1=load_data(1,'test')
Xtr2=load_data(2,'test')



In [14]:
np.shape(phi(X))

(2000, 84)

In [15]:
fixedsub_kernel(X[0:1],X[3:4],3)

NameError: name 'fixedsub_kernel' is not defined

In [16]:
# 
def generate_uplets(alphabet,n,aux):
    if n==0:
        return aux
    else:
        tmp = []
        for a in (aux):
            for l in alphabet:
                tmp.append(l+a)
            
        return generate_uplets(alphabet,n-1,tmp)
    
    

def spectrum(x,k=3):
    """
    Computes the spectrum kernel of the vector x
    Parameters
    ----------
    x : array containing the sequences in string format.
        Second dimension must be 1
    k : length of substrings to consider
    Returns
    -------
    phix : spectrum representation of x. Array containing for each word
            of size k the number of occurences of the word in x
    """
    n = np.shape(x)[0]
    k_uplets = generate_uplets('ACTG',k,[''])
    
    occs = np.zeros((n,len(k_uplets)))
    for i,sequence in enumerate(x):
        for j,s in enumerate(k_uplets):
            occs[i,j]=sequence.count(s)
    
    return occs

def spectrum_kernel(x1,x2,k=3):
    """
    Computes the spectrum kernel between x1 and x2
    """
    
    tmp1,tmp2 = x1.reshape(np.shape(x1)[0]),x2.reshape(np.shape(x2)[0])
    s1 , s2 = spectrum(tmp1,k),spectrum(tmp2,k)
    
    
    return np.dot(s1,s2.T)

In [17]:
np.shape(X)

(2000,)

In [18]:
np.dot(X,X.T)

TypeError: can't multiply sequence by non-int of type 'str'

In [19]:
reload(Classifier)
clf = Classifier.SVM()

clf.setKernel(name='manual',f=spectrum_kernel)


def fetch_data(Xpath,Ypath=None):
    data = pd.read_csv(Xpath,header=-1)
    X=data.as_matrix()
    X = X.reshape(np.shape(X)[0])
    
    # If there are no labels
    if Ypath==None:
        return X
        
    else:
        labels = pd.read_csv(Ypath)
        Y=labels['Bound'].as_matrix()
        return X,Y
    
    

X0,Y0 = fetch_data(data_dir+'Xtr0.csv',data_dir+'Ytr0.csv')
X1,Y1 = fetch_data(data_dir+'Xtr1.csv',data_dir+'Ytr1.csv')
X2,Y2 = fetch_data(data_dir+'Xtr2.csv',data_dir+'Ytr2.csv')
Xtr = np.concatenate([X0,X1,X2])
Ytr = np.concatenate([Y0,Y1,Y2])
Xtr=Xtr.reshape([np.shape(Xtr)[0],1])

X0 = fetch_data(data_dir+'Xte0.csv')
X1 = fetch_data(data_dir+'Xte1.csv')
X2 = fetch_data(data_dir+'Xte2.csv')
Xte = np.concatenate([X0,X1,X2])
Xte=Xte.reshape([np.shape(Xte)[0],1])

Ktr = spectrum_kernel(Xtr,Xtr)
clf.set_gram(Ktr)

clf.train(Xtr,Ytr)

Yte = svm.predict(Xte)


len(Yte)

df = pd.DataFrame(Yte,columns =  ['Bound'])



df.head()

df.to_csv('Yte.txt',index_label='Id')

TypeError: __init__() missing 1 required positional argument: 'kernel'

In [189]:
X_spec = spectrum(X,3)

perm = np.random.permutation(2000)
tr_idx,test_idx = perm[:1500],perm[1500:]

from sklearn.svm import SVC

clf = SVC()

Xtr,Xte,Ytr,Yte = X_spec[tr_idx],X_spec[test_idx],Y[tr_idx],Y[test_idx]

clf.fit(Xtr,Ytr)

clf.score(Xte,Yte)

0.48999999999999999

In [190]:
# crossvalidation scores

def cross_val(clf,X,Y,n_splits=10):
    """
    Computes the scores of the classifier on cross validation dividing the dataset in n_split parts
    """
    
    n,p = np.shape(X)
    
    
    width = n//n_splits
    perm = np.random.permutation(n)
    splits = [perm[k*width:(k+1)*width] for k in range(n_splits)]
    scores = []
    for k in range(n_splits):
        tr_idx=splits[k]
        
        test_idx=np.concatenate([splits[i] for i in range(n_splits) if i!=k])
        
        Xtr,Xte,Ytr,Yte = X_spec[tr_idx],X_spec[test_idx],Y[tr_idx],Y[test_idx]

        clf.fit(Xtr,Ytr)
        scores.append(clf.score(Xte,Yte))
    return scores

cross_val(SVC(),X_spec,Y,n_splits=2)
          

[0.49533333333333335, 0.49533333333333335]

In [ ]:
import time


In [79]:
np.shape(X)

(6000, 1)

### Mismatch kernel

In [21]:
def mismatch(sequence,k,m):
    """
    Computes the mismatch kernel of the vector x, allowing up to m mismatches 
    Parameters
    ----------
    x : sequence
    k : length of substrings to consider
    Returns
    -------
    phix : spectrum representation of x. Array containing for each word
            of size k the number of occurences of the word in x
    """
    k_uplets = generate_uplets('ACTG',k,[''])
    occs = np.zeros(len(k_uplets))
    for i,s in enumerate(k_uplets):
        
        occs[i]=sequence.count(s)
    return occs




In [25]:
seq = X[0]
seq

'TCCTCAACTTTTATTGGGCCGCTGTGGCACCAGAATCTACGAATGGCGCCCTCTAGAGTTGTGTAAAGAAGTGGCGTCACCTCATTATAAATAAAAGGTTG'

In [ ]:
def B(n,x1,x2,lam):
    if n==0 or min(len(x1),len(x2))<n:
        return 0
    else:
        print(n)
        a=x1[-1]
        b=x2[-1]
        x1_new=x1[:-1]
        x2_new=x2[:-1]
        if a==b:
            return lam*B(n,x1_new,x2,lam)+lam*B(n,x1,x2_new,lam)-lam**2*B(n,x1_new,x2_new,lam) + lam**2*B(n-1,x1_new,x2_new,lam)
        else:
            return lam*B(n,x1_new,x2,lam)+lam*B(n,x1,x2_new,lam)-lam**2*B(n,x1_new,x2_new,lam) 
        

In [17]:
seq=X[0]
seq[0:3]

#substring_kernel(seq,seq)

test=np.zeros(100)
test is None


False

### All Substring Kernel
Representation indexed by all the substrings of the string.
The value of a given component is the number of occurence of the associated substring in the string.


In [1]:
def allsub(s,t):
    """ 
    Computes the all-subsequence kernel of the strings s and t.
    The returned value is the number of subsequence, including gaps, that 
    appear in s and t. 
    Parameters
    ----------
    s,t: Input strings
    tab: Dynamic programming table
    Returns
    -------
    The kernel value
    """
    # Initialize dynamic programming table
    n,m=len(s),len(t)
    tab = np.ones((n+1,m+1))
    P = np.zeros(m+1)
    
    for i in range(1,n+1):
        last = 0
        for k in range(1,m+1):
            P[k]=P[last]
            if(t[k-1]==s[i-1]):
                P[k]=P[last]+tab[i-1][k-1]
                last=k
        
        for k in range(1,m+1):
            tab[i][k]=tab[i-1][k] + P[k]
            
    return tab[-1,-1]

def allsub_kernel(s,t):
    return allsub(s,t)/np.sqrt(allsub(s,s)*allsub(t,t))

allsub(X[0],X[0])

NameError: name 'X' is not defined

### Fixed-length Substring Kernel
Representation indexed by all the substrings of the string.
The value of a given component is the number of occurence of the associated substring in the string.


In [14]:
def fixedsub(s,t,p):
    n,m=len(s),len(t)
    tab = np.ones((n+1,m+1))

    for l in range(1,p+1):
        old_tab = tab
        tab[0,:]=np.ones(m+1)
        for i in range(1,n-p+l):
            last=0
            P=np.zeros(m+1)
            for k in range(1,m+1):
                P[k]=P[last]
                if t[k-1]==s[i-1]:
                    P[k]=P[last]+old_tab[i-1,k-1]
                    last=k
            for k in range(1,m+1):
                tab[i,k]=tab[i-1,k]+P[k]
    
    return tab[n-1,m-1]
         
def fixedsub_kernel(s,t,p):
    return fixedsub(s,t,p)/np.sqrt(fixedsub(s,s,p)*fixedsub(s,s,p))
        
fixedsub(X[0],X[0],1)

3.6296884485463954e+40

In [125]:
def substring(s,t,p=3,lam=0.7):
    """ 
    Computes the substring kernel between s and t with subsequence length p and gap-weight lam
    
    """
    n,m=len(s),len(t)
    dps=np.zeros((n+1,m+1))
    
    for i in range(1,n+1):
        for j in range(1,m+1):
            if(s[i-1]==t[i-1]):
                dps[i,j]=lam**2
    dp=np.zeros((n+1,m+1))
    for l in range(2,p+1):
        ker=0
        for i in range(1,n):
            for j in range(1,m):
                dp[i,j]=dps[i,j]+lam*dp[i-1,j]+lam*dp[i,j-1]-lam**2*dp[i-1,j-1]
                if(s[i-1]==t[j-1]):
                    dps[i,j]=lam**2*dp[i-1,j-1]
                    ker=ker+dps[i,j]
    return ker

def substring_normalized(s,t,p=3,lam=0.7):
    return substring(s,t,p,lam)/np.sqrt(substring(s,s,p,lam)*substring(t,t,p,lam))


def substring_gram(X,p=3,lam=0.7):
    Xa=np.ravel(X) # make sure every component of X is a string
    n=np.shape(X)[0]
    K=np.zeros((n,n))
    for i in range(n):
        for j in range(i,n):
            K[i,j]=substring_normalized(X[i],X[j],p,lam)
    K=(K+K.T)*0.5
    return K
substring_normalized(X[0],X[0])

1.0

In [8]:
X,Y=load_data(0,'train')


In [165]:
import time
t0= time.time()
substring(X[0],X[1])
print('done in ',time.time()-t0)

done in  0.05414867401123047


In [166]:
K=substring_gram(X[0:10])

In [187]:
def spectrum_trie(s,t,p):
        Ls={}
        Lt={}
        Ls['']=[(s[i:i+p],0) for i in range(len(s)-p+1)]
        Lt['']=[(t[i:i+p],0) for i in range(len(t)-p+1)]
        kern=0

        def processnode(v,depth,p,Ls,Lt,alphabet='ACTG'):
            """
            v:current node
            depth:current depth in the trie
            p:length of the substrings to be considered
            Ls,Lt : dictionaries containing the lists associated with the visited nodes
            kern: value of the kernel
            """
            Ls_new=Ls
            Lt_new=Lt
            if (depth==p):
                kern=kern+len(Ls[v])*len(Lt[v])
            else:
                
                if(len(Ls[v])*len(Lt[v])!=0):
                    Lsv_copy=Ls[v]
                    while len(Lsv_copy)!=0:
                        u,i=Lsv_copy.pop()
                        key=v+u[i+1]
                        if not (Ls_new.__contains__(key)):
                            Ls_new[key]=[]
                        Ls_new[key].append((u,i+1))

                    Ltv_copy=Lt[v]

                    while len(Ltv_copy)!=0:
                        u,i=Ltv_copy.pop()
                        key=v+u[i+1]
                        if not (Lt_new.__contains__(key)):
                            Lt_new[key]=[]
                        Lt_new[key].append((u,i+1))
                    
                    for a in alphabet:
                        processnode(v+a,depth+1,Ls_new,Lt_new,alphabet)

        processnode('',0,p,Ls,Lt)
        return kern
spectrum_trie(X[0],X[0],3)

TypeError: string indices must be integers

In [196]:
import os
os.listdir('kernels/src')

['.ipynb_checkpoints',
 '__pycache__',
 'baselines.py',
 'challengeFunctions.py',
 'Classifier.py',
 'kernels.py',
 'main.py',
 'TEST.ipynb',
 'Yte.txt']

In [201]:
from kernels.src.kernels import spectrum,generate_uplets
import pandas as pd
from importlib import reload


In [202]:
help(spectrum)

Help on function spectrum in module kernels.src.kernels:

spectrum(x, k=3)
    Computes the spectrum kernel of the vector x
    Parameters
    ----------
    x : array containing the sequences in string format.
    
    k : length of substrings to consider
    Returns
    -------
    phix : spectrum representation of x. Array containing for each word
            of size k the number of occurences of the word in x



In [197]:
# Data directory
data_dir = '../../data_kernel/'

# load training data
def fetch_data(Xpath,Ypath=None):
    """
    Loads the data and labels , stores them into numpy arrays.
    If no Ypath is given, then it is assumed that only a test set is wanted
    The returned X is a 1-dimensional array containing strings
    """
    data = pd.read_csv(Xpath,header=-1)
    X=data.as_matrix()
    X = X.reshape(np.shape(X)[0])
    
    # If there are no labels
    if Ypath==None:
        return X
        
    else:
        labels = pd.read_csv(Ypath)
        Y=labels['Bound'].as_matrix()
        return X,Y
    
    

def plot_acc(Xdir,Ydir,lambs=[0.001,0.01,0.1,1,15,50, 100],k=4):
    X,Y=fetch_data(Xdir,Ydir)
    X=spectrum(X,k)
    Xtrain, Y_train, Xtest, Y_test=cf.splitdata(X,Y)


    train_acc = []
    test_acc = []
    best_test_acc=0
    best_lam=lambs[0]
    for lamb in lambs:
        svm = Classifier.SVM("polynomial")
        svm.lamb = lamb

        Y_train_svm = 2*(Y_train - 1/2)
        Y_test_svm = 2*(Y_test - 1/2)
        svm.train(Xtrain, Y_train_svm)
        y_pred = svm.predict(Xtrain)
        y_pred = y_pred > 0
        train_acc.append(cf.classification_accuracy(Y_train, y_pred))
        y_pred = svm.predict(Xtest)
        y_pred = y_pred > 0
        test_acc.append(cf.classification_accuracy(Y_test, y_pred))
        
        print(test_acc[-1])
        if(test_acc[-1]>best_test_acc):
            best_test_acc=test_acc[-1]
            best_lam=lamb
            
    plt.semilogx(lambs,train_acc)
    plt.semilogx(lambs,test_acc)
    plt.legend(["Training",'Test'])
    plt.show()
    return best_lam,best_test_acc

best_k=3
best_acc=0
best_lam=3

for k in [3,4,5,6,7]:    
    bl,ba = plot_acc(data_dir+'Xtr0.csv',data_dir+'Ytr0.csv',[0.001,0.01,1,10,100],k)
    if (ba>best_acc):
        best_acc=ba
        best_lam=bl
        best_k=k

best_lam,best_k,best_acc

test_acc


svm = Classifier.SVM("gaussian")
svm.lamb = lamb

Y_train_svm = 2*(Y_train - 1/2)
Y_test_svm = 2*(Y_test - 1/2)
svm.train(Xtrain, Y_train_svm)
y_pred = svm.predict(Xtrain)
y_pred = y_pred > 0
train_acc.append(cf.classification_accuracy(Y_train, y_pred))
y_pred = svm.predict(Xtest)
y_pred = y_pred > 0
test_acc.append(cf.classification_accuracy(Y_test, y_pred))

def fetch_data(Xpath,Ypath=None):
    data = pd.read_csv(Xpath,header=-1)
    X=data.as_matrix()
    X = X.reshape(np.shape(X)[0])
    
    # If there are no labels
    if Ypath==None:
        return X
        
    else:
        labels = pd.read_csv(Ypath)
        Y=labels['Bound'].as_matrix()
        return X,Y

    

X0,Y0 = fetch_data(data_dir+'Xtr0.csv',data_dir+'Ytr0.csv')
X0=spectrum(X0)
X1,Y1 = fetch_data(data_dir+'Xtr1.csv',data_dir+'Ytr1.csv')
X1=spectrum(X1)
X2,Y2 = fetch_data(data_dir+'Xtr2.csv',data_dir+'Ytr2.csv')
X2=spectrum(X2)

Xtr = np.vstack([X0,X1,X2])
Ytr = np.hstack([Y0,Y1,Y2])
Ytr = 2*(Ytr-1/2)


svm=Classifier.SVM("polynomial")
svm.lam=10



data_dir='../../data_kernel/'

#0
k0=6
X0,Y0 = fetch_data(data_dir+'Xtr0.csv',data_dir+'Ytr0.csv')
Y0=2*(Y0-1/2)
X0=spectrum(X0,k0)
svm = Classifier.SVM("polynomial")
svm.lamb = 0.1
svm.train(X0, Y0)

Xte0 = spectrum(fetch_data(data_dir+'Xte0.csv'),k0)

Yte0=svm.predict(Xte0)

Yte0= Yte0 > 0

#1
k1=5
X1,Y1 = fetch_data(data_dir+'Xtr1.csv',data_dir+'Ytr1.csv')
Y1=2*(Y1-1/2)
X1=spectrum(X1,k1)

svm = Classifier.SVM("polynomial")
svm.lamb = 0.1
svm.train(X1, Y1)

Xte1 = spectrum(fetch_data(data_dir+'Xte1.csv'),k1)

Yte1=svm.predict(Xte1)
Yte1= Yte1>0

#2
k2=6
X2,Y2 = fetch_data(data_dir+'Xtr2.csv',data_dir+'Ytr2.csv')
Y2=2*(Y2-1/2)
X2=spectrum(X2,k2)
svm = Classifier.SVM("polynomial")
svm.lamb = 100
svm.train(X2, Y2)

Xte2 = spectrum(fetch_data(data_dir+'Xte2.csv'),k2)
Yte2=svm.predict(Xte2)
Yte2= Yte2>0


# Save result
Yte=np.concatenate([Yte0,Yte1,Yte2])
Yted=(Yte.astype(int))
df = pd.DataFrame(Yted,columns =  ['Bound'])

df.head()
df.to_csv('Yte.txt',index_label='Id')

Yte=np.concatenate([Yte0,Yte1,Yte2])

Yted=(Yte.astype(int))

np.sum(Yted)

df = pd.DataFrame(Yted,columns =  ['Bound'])

df.head()
df.to_csv('Yte.txt',index_label='Id')

ModuleNotFoundError: No module named 'kernel'